# Data Analysis

## Solutions object
Obtain the Hidden Sector from: https://zenodo.org/record/5526707#.YzwIANJBzeQ and add the missing the column `Hidden Sector` of the following data scheme

![img](./squeme.svg)

### Example of entry

In [3]:
from IPython.display import JSON 
import pandas as pd

In [3]:
JSON({'A':1})

<IPython.core.display.JSON object>

In [4]:
JSON(([{'l':[1,3],'k':[2],
       'solution':[1,2,-3,4,5],'gcd':1,'n':5,
      'hidden':[{'ψ':[(1,5),(2,4),(-3,)],
                 'S':6}
               ]
             }] 
            )
    )


<IPython.core.display.JSON object>

In [5]:
import pandas as pd

In [6]:
pd.DataFrame([{'l':[1,3],'k':[2],
       'solution':[1,2,-3,4,5],'gcd':1,'n':5,
      'hidden':[{'ψ':[(1,5),(2,4),(-3,)],
                 'S':6}
               ]
             }] 
            )

,l,k,solution,gcd,n,hidden
0,"[1, 3]",[2],"[1, 2, -3, 4, 5]",1,5,"[{'ψ': [(1, 5), (2, 4), (-3,)], 'S': 6}]"


### Read the full dataset

In [4]:
df=pd.read_json('https://zenodo.org/record/5526707/files/solutions.json?download=1')

In [8]:
df['zmax']=df['solution'].str[-1].abs()
df=df.sort_values(['n','zmax']).reset_index(drop=True)

#How many solutions with n=12 and z_max <= 20?
df[ (df['n']==12) & (df['zmax']<=20) ].shape

(16862, 6)

### Add the new column with the list of hidden sectors

In [5]:
import itertools

In [6]:
list(itertools.combinations_with_replacement([1,2],2))

[(1, 1), (1, 2), (2, 2)]

In [40]:
def get_hidden_sector(l):
    '''
    Extract massive pairs without massles fermions from l
    '''
    sltns=[]
    #ps = [ (x,y) for x in l for y in l if x<=y]
    ps = list(itertools.combinations_with_replacement(set(l),2))
    Ss = set(abs(sum(x)) for x in ps)
    # capture all the possible scalar, Ss, values    
    for s in set(Ss):
        msv =([p for p in  ps if abs(sum(p))==s])
        # flatten the list
        massives=set([x for sublist in msv for x in sublist])
        massless=set(l).difference(massives)
        if not massless:
            sltn={}
            sltn['S']=s
            sltn['ψ']=msv
            #================================
            #Quality checks on repeated integers:
            rptd=set([x for x in l if l.count(x)>1])
            majoranas=[m[0] for m in msv if m[0]==m[1]]
            #remove majoranas from the analysis
            rptd=rptd.difference(majoranas)
            #Diracs need come from repeated quiral fields (a,b) → [a,a,b,b] 
            diracs=[d for d in itertools.combinations(rptd,2) if 
                    abs(sum(d))==s and l.count(d[0])==l.count(d[1])]
            #if diracs and not set([x for sublist in diracs for x in sublist])==rptd
            if not rptd == set([x for sublist in diracs for x in sublist]):
                sltn={}
            #======================================            
            if sltn:
                sltns.append(sltn)
            
    return sltns

assert get_hidden_sector([1,2,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,2,-3,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,1,2,-3,-3,4,5,5])[0].get('S')==6
#aψ1ψ1+bψ1ψ2+cψ1,ψ3 → https://www.wolframalpha.com/input?i=Rank%20{{a,b,c},{b,0,0},{c,0,0}}
assert get_hidden_sector([1,2,-3,4,5,9,9])==[]
get_hidden_sector([1,1,2,-3,4,5])
#aψ1ψ2+bψ1ψ3 → https://www.wolframalpha.com/input?i=rank+{{0,a,b},{a,0,0},{b,0,0}}
assert get_hidden_sector([1,1,2,-3,4,5])==[]
assert get_hidden_sector([1,1,2,-3,-3,4,5])==[]
assert get_hidden_sector([1,2,-3,4,5,8])==[]
assert get_hidden_sector([1,2,-3,4,5,8,8])==[]
assert get_hidden_sector([])==[]
assert get_hidden_sector([1, 1, 1, 1, 1, -2, -2, -2, -2, 3])==[]

In [38]:
df['hidden']=df['solution'].apply(get_hidden_sector)

In [42]:
hs=df[df['hidden'].apply(len)>0].reset_index(drop=True)#.iloc[0].to_dict()

In [43]:
hs.shape

(6029, 7)

## Ask question to the solutions object

* Test the solution
* Which is the minimum hidden sector and with the least charge in absolute value
* Check for multigenerational solution
* Check for unconditional stability
* Solutions with active symmetry
* How many solutions with n=12 and zmax<=20
* Check for Majorana and Dirac fermions

In [44]:
pd.set_option('display.max_colwidth',200)

In [45]:
# What is the minimum n?
hs['n'].min()

7

In [46]:
# What is the minimum z_max?
hs['zmax'].min()

5

In [47]:
hs[hs['zmax']==hs['zmax'].min()]

,l,k,solution,gcd,n,zmax,hidden
1374,"[-1, -2, -1, 0, 1]","[-3, -2, -3, -2, 0]","[1, 1, -2, -2, 3, 3, 3, -4, -4, -4, 5]",1,11,5,"[{'S': 1, 'ψ': [(1, -2), (3, -4), (3, -2), (5, -4)]}]"


In [48]:
hs[:3]

,l,k,solution,gcd,n,zmax,hidden
0,"[-2, 0]","[-1, -2, -3]","[3, 5, -8, 9, -10, -14, 15]",1,7,15,"[{'S': 5, 'ψ': [(3, -8), (5, -10), (9, -14), (15, -10)]}]"
1,"[2, 0]","[-1, -2, -3]","[5, 6, 7, -10, -11, -12, 15]",1,7,15,"[{'S': 5, 'ψ': [(5, -10), (6, -11), (7, -12), (15, -10)]}]"
2,"[-1, 2, -2]","[-7, 4, 0]","[1, 2, 2, 2, -3, -5, -6, 7]",1,8,7,"[{'S': 4, 'ψ': [(1, -5), (2, 2), (2, -6), (7, -3)]}]"


In [49]:
# How many solutions have several dark sectors?
hs[hs['hidden'].apply(len)>1].shape

(527, 7)

In [50]:
hs[hs['hidden'].apply(len)>2].shape

(0, 7)

In [51]:
# What is the minimum number of DM particles?
hs['hidden'].str[0].str['ψ'].apply(len).min()

3

In [53]:
hs[hs['hidden'].str[0].str['ψ'].apply(len)==hs['hidden'].str[0].str['ψ'].apply(len).min()][:1]

,l,k,solution,gcd,n,zmax,hidden
3,"[-1, 0, 1]","[-4, -2, 2]","[1, -2, -2, 4, 5, -7, -7, 8]",1,8,8,"[{'S': 9, 'ψ': [(1, 8), (4, 5), (-7, -2)]}]"


### Same questions for SM solutions

In [54]:
df3=df[df['solution'].apply(lambda l: 3 in [l.count(x) for x in l])].reset_index(drop=True)

In [55]:
def get_dark(l,idx=0):
    #TODO,
    xxx=list(set([x for x in l if l.count(x)==3]))
    if len(xxx)>=idx+1:
        xx=xxx[idx]
        return [x for x in l if x!=xx]
    else:
        return []
assert get_dark([1,1,2,2,2,3] )==[1, 1, 3]

In [56]:
df3['hidden']=df3['solution'].apply(get_dark).apply(get_hidden_sector)

In [57]:
hs3=df3[df3['hidden'].apply(len)>0].reset_index(drop=True)
hs3.shape

(124, 7)

In [58]:
# idxmax==1
df3['hidden']=df3['solution'].apply(lambda l: get_dark(l,idx=1)).apply(get_hidden_sector)
hs3=pd.concat((hs3,df3[df3['hidden'].apply(len)>0])).reset_index(drop=True)
hs3.shape

(129, 7)

In [59]:
# idxmax==1
df3['hidden']=df3['solution'].apply(lambda l: get_dark(l,idx=2)).apply(get_hidden_sector)
hs3=pd.concat((hs3,df3[df3['hidden'].apply(len)>0])).reset_index(drop=True)
hs3.shape

(129, 7)

In [60]:
hs3['zmax'].min()

9

In [61]:
hs3[:1]

,l,k,solution,gcd,n,zmax,hidden
0,"[1, 0, -2]","[-3, -1, 1]","[3, 3, 3, 5, -7, -9, -11, 13]",2,8,13,"[{'S': 18, 'ψ': [(5, 13), (-11, -7), (-9, -9)]}]"


In [62]:
hs3[hs3['hidden'].apply(len)>1].shape

(4, 7)

In [63]:
hs3[hs3['hidden'].apply(len)>2].shape

(2, 7)

In [64]:
hs3[hs3['hidden'].apply(len)>3].shape

(0, 7)

In [65]:
hs3['hidden'].str[0].str['ψ'].apply(len).min()

2

In [67]:
hs3[hs3['hidden'].str[0].str['ψ'].apply(len)==hs3['hidden'].str[0].str['ψ'].apply(len).min()]

,l,k,solution,gcd,n,zmax,hidden
4,"[-2, 0, 2]","[-1, 1, 0, -1]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",1,9,10,"[{'S': 9, 'ψ': [(1, -10), (-5, -4)]}]"
6,"[-1, 0, 1]","[-1, -3, 0, -1]","[1, 1, -3, -8, 11, 11, 11, -12, -12]",1,9,12,"[{'S': 11, 'ψ': [(-8, -3), (1, -12)]}]"
8,"[-1, 0, -1, 1]","[-3, 0, -1, 0]","[4, -5, -5, -6, -6, 11, 11, 11, -15]",1,9,15,"[{'S': 11, 'ψ': [(-15, 4), (-6, -5)]}]"
16,"[-1, 0, 1]","[-1, -5, 0, 5]","[2, 2, -3, 15, 15, -17, -17, -17, 20]",3,9,20,"[{'S': 17, 'ψ': [(2, 15), (20, -3)]}]"
18,"[-1, 0, -1, 1]","[-3, 0, -3, 0]","[2, 2, -7, -12, 19, 19, 19, -21, -21]",1,9,21,"[{'S': 19, 'ψ': [(-7, -12), (2, -21)]}]"
19,"[-1, 0, 1]","[-1, 1, 0, -7]","[3, 3, -4, 14, 14, -17, -17, -17, 21]",3,9,21,"[{'S': 17, 'ψ': [(3, 14), (-4, 21)]}]"
24,"[1, 0, 1, -1]","[-3, 0, -3, 0]","[4, 4, -5, 15, 15, -19, -19, -19, 24]",1,9,24,"[{'S': 19, 'ψ': [(24, -5), (4, 15)]}]"


### Extend the data scheme with number of generations

In [68]:
def get_Dirac_and_Majorana_generations(l,h):
    g=[]
    dgmax=0
    mgmax=0
    for idx in range(len(h)):
        for x,y in h[idx].get('ψ'):
            if x!=y and l.count(x)>dgmax and l.count(y)==l.count(x):
                    dgmax=l.count(x)
            if x==y and l.count(x)>mgmax:
                    mgmax=l.count(x)
        g.append({'DG':dgmax,'NG':mgmax})
    return g
l=[2, 2, -3, 15, 15, -17, -17, -17, 20]
h=[{'S': 17, 'ψ': [(2, 15), (20, -3)]}]
assert get_Dirac_and_Majorana_generations(l,h)==[{'DG': 2, 'NG': 0}]

In [69]:
hs['Generations']=hs.apply(lambda row: get_Dirac_and_Majorana_generations(row['solution'],row['hidden']),axis='columns')

In [70]:
hs3['Generations']=hs3.apply(lambda row: get_Dirac_and_Majorana_generations(row['solution'],row['hidden']),axis='columns')

In [71]:
hs['Gmax']=-hs['Generations'].str[0].apply(lambda d: sum(d.values()))

In [72]:
hs=hs.sort_values(['n','Gmax']).reset_index(drop=True)

In [73]:
hs[hs['n']==7]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
0,"[-2, 0]","[-1, -2, -3]","[3, 5, -8, 9, -10, -14, 15]",1,7,15,"[{'S': 5, 'ψ': [(3, -8), (5, -10), (9, -14), (15, -10)]}]","[{'DG': 1, 'NG': 0}]",-1
1,"[2, 0]","[-1, -2, -3]","[5, 6, 7, -10, -11, -12, 15]",1,7,15,"[{'S': 5, 'ψ': [(5, -10), (6, -11), (7, -12), (15, -10)]}]","[{'DG': 1, 'NG': 0}]",-1


In [76]:
hs[hs['n']==8][:6]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
2,"[-1, 2, -2]","[-7, 4, 0]","[1, 2, 2, 2, -3, -5, -6, 7]",1,8,7,"[{'S': 4, 'ψ': [(1, -5), (2, 2), (2, -6), (7, -3)]}]","[{'DG': 1, 'NG': 3}]",-4
3,"[1, 0, -2]","[-3, -1, 1]","[3, 3, 3, 5, -7, -9, -11, 13]",2,8,13,"[{'S': 6, 'ψ': [(3, 3), (3, -9), (5, -11), (13, -7)]}]","[{'DG': 1, 'NG': 3}]",-4
4,"[0, -1, -2]","[-1, -6, -5, -6]","[1, -6, -6, -6, 7, 11, 18, -19]",2,8,19,"[{'S': 12, 'ψ': [(1, 11), (7, -19), (18, -6), (-6, -6)]}]","[{'DG': 1, 'NG': 3}]",-4
5,"[3, 0, -5]","[-1, 1, -1]","[4, 4, 4, 11, -12, -13, -19, 21]",8,8,21,"[{'S': 8, 'ψ': [(4, 4), (4, -12), (11, -19), (-13, 21)]}]","[{'DG': 1, 'NG': 3}]",-4
6,"[0, -2, 0]","[-3, -1, 1, -1]","[5, 5, 5, -15, 19, -21, -29, 31]",2,8,31,"[{'S': 10, 'ψ': [(-29, 19), (5, 5), (5, -15), (-21, 31)]}]","[{'DG': 1, 'NG': 3}]",-4
7,"[0, 3, -1]","[-2, -1, 3, -1]","[1, -2, -3, 6, 6, -9, -10, 11]",1,8,11,"[{'S': 12, 'ψ': [(1, 11), (6, 6), (-10, -2), (-9, -3)]}]","[{'DG': 1, 'NG': 2}]",-3


In [79]:
hs[hs['n']==9][:3]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
1357,"[-2, 0, 1, 3]","[-1, 0, -1, 3]","[1, 1, 2, 2, -6, 9, -10, -12, 13]",1,9,13,"[{'S': 3, 'ψ': [(1, 2), (9, -12), (9, -6), (13, -10)]}]","[{'DG': 2, 'NG': 0}]",-2
1358,"[1, 2, 0, -1]","[-5, 2, 0, -5]","[3, -8, 10, 10, 11, -15, -15, -16, 20]",1,9,20,"[{'S': 5, 'ψ': [(3, -8), (10, -15), (11, -16), (-15, 20)]}]","[{'DG': 2, 'NG': 0}]",-2
1359,"[-2, 3, 2]","[-3, -8, -2, 1]","[1, 4, -10, -10, -12, 15, 15, 17, -20]",6,9,20,"[{'S': 5, 'ψ': [(1, 4), (-20, 15), (15, -10), (17, -12)]}]","[{'DG': 2, 'NG': 0}]",-2


In [80]:
hs[hs['n']==11][:4]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
1374,"[-2, -1, -2, -1, 3]","[-3, -1, 2, -1, 3]","[1, 1, 1, 1, -3, 5, -6, 8, -9, -10, 11]",2,11,11,"[{'S': 2, 'ψ': [(1, 1), (1, -3), (5, -3), (8, -10), (8, -6), (11, -9)]}]","[{'DG': 1, 'NG': 4}]",-5
1375,"[-4, 1, 0, 4]","[-1, -3, 1, 0, 2]","[1, 4, 4, 4, 4, -8, -9, 16, -17, -24, 25]",4,11,25,"[{'S': 8, 'ψ': [(1, -9), (4, 4), (-24, 16), (-17, 25), (16, -8)]}]","[{'DG': 1, 'NG': 4}]",-5
1376,"[-1, 0, 1, 0, 2]","[-3, -2, -1, -2, 0]","[1, 1, -2, -2, 4, 4, -6, 7, -8, -9, 10]",2,11,10,"[{'S': 2, 'ψ': [(1, 1), (4, -6), (4, -2), (7, -9), (10, -8)]}]","[{'DG': 2, 'NG': 2}]",-4
1377,"[1, 0, 3, 0, -2]","[-2, -1, -2, -3, 0]","[1, 3, 3, -6, -6, -7, -10, 12, 12, 16, -18]",2,11,18,"[{'S': 6, 'ψ': [(1, -7), (3, 3), (12, -18), (12, -6), (16, -10)]}]","[{'DG': 2, 'NG': 2}]",-4


In [81]:
hs[hs['n']==12][:4]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
1800,"[1, 0, -2, 0, -2]","[-3, -1, 1, -1, 1]","[1, 1, 7, 7, 7, 7, 7, -11, -15, -15, -21, 25]",2,12,25,"[{'S': 14, 'ψ': [(1, -15), (7, 7), (7, -21), (-11, 25)]}]","[{'DG': 2, 'NG': 5}]",-7
1801,"[-1, 0, -3, 0, -1]","[-1, 1, -1, -3, -1]","[5, 5, 7, 7, 7, 7, 7, -15, -19, -19, -21, 29]",2,12,29,"[{'S': 14, 'ψ': [(5, -19), (7, 7), (7, -21), (-15, 29)]}]","[{'DG': 2, 'NG': 5}]",-7
1802,"[2, 0, -1, 0, 1]","[-3, -2, -3, -2, -1]","[1, -2, -2, -3, 4, 4, 4, 4, -5, -6, -6, 7]",2,12,7,"[{'S': 8, 'ψ': [(1, 7), (4, 4), (-6, -2), (-5, -3)]}]","[{'DG': 2, 'NG': 4}]",-6
1803,"[-1, 1, -2, -1, 1]","[-2, 0, -1, -3, 0]","[1, 1, 1, 1, 1, 2, -3, -4, 5, -6, -7, 8]",1,12,8,"[{'S': 2, 'ψ': [(1, 1), (1, -3), (2, -4), (5, -7), (5, -3), (8, -6)]}]","[{'DG': 1, 'NG': 5}]",-6


In [82]:
hs3['Gmax']=-hs3['Generations'].str[0].apply(lambda d: sum(d.values()))
hs3=hs3.sort_values(['n','Gmax']).reset_index(drop=True)

In [83]:
hs3[hs3['n']==8][:1]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
0,"[1, 0, -2]","[-3, -1, 1]","[3, 3, 3, 5, -7, -9, -11, 13]",2,8,13,"[{'S': 18, 'ψ': [(5, 13), (-11, -7), (-9, -9)]}]","[{'DG': 1, 'NG': 1}]",-2


In [84]:
hs3[hs3['n']==9][:4]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
3,"[-2, -1, 0]","[-1, -3, -5, -3]","[5, 6, 6, 6, -8, -9, -9, -10, 13]",2,9,13,"[{'S': 18, 'ψ': [(5, 13), (-10, -8), (-9, -9)]}]","[{'DG': 1, 'NG': 2}]",-3
4,"[1, -2, -1]","[-4, -9, -5, -3]","[1, -2, 6, 6, 6, -9, -9, -16, 17]",6,9,17,"[{'S': 18, 'ψ': [(1, 17), (-16, -2), (-9, -9)]}]","[{'DG': 1, 'NG': 2}]",-3
5,"[-2, 0, 2]","[-1, 1, 0, -1]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",1,9,10,"[{'S': 9, 'ψ': [(1, -10), (-5, -4)]}]","[{'DG': 2, 'NG': 0}]",-2
6,"[2, -2, 1, 3]","[-5, -4, -3, -1]","[1, -2, -2, -2, 5, -7, 8, 9, -10]",1,9,10,"[{'S': 2, 'ψ': [(1, 1), (5, -7), (8, -10), (9, -7)]}]","[{'DG': 1, 'NG': 1}]",-2


In [85]:
hs3[hs3['n']==10][:5]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
44,"[-4, -1, 0, 1]","[-4, 1, -1, -3, 1]","[1, 1, 1, 2, 2, -7, 11, -12, -15, 16]",3,10,16,"[{'S': 4, 'ψ': [(2, 2), (11, -15), (11, -7), (16, -12)]}]","[{'DG': 1, 'NG': 2}]",-3
45,"[-3, 3, 0, -6]","[-3, -2, 3, 4]","[3, 3, 3, -4, 9, 9, -10, -14, -27, 28]",18,10,28,"[{'S': 18, 'ψ': [(-27, 9), (9, 9), (28, -10), (-14, -4)]}]","[{'DG': 1, 'NG': 2}]",-3
46,"[-1, 0, -4, -3]","[-3, 1, -1, -5]","[1, -3, -3, -3, -9, -9, 11, 17, 27, -29]",2,10,29,"[{'S': 18, 'ψ': [(1, 17), (-29, 11), (-9, -9), (-9, 27)]}]","[{'DG': 1, 'NG': 2}]",-3
47,"[0, 1, 0, -2]","[-1, -3, 3, -1, -3]","[1, -5, -7, 9, -11, -13, 15, 15, 15, -19]",4,10,19,"[{'S': 18, 'ψ': [(1, -19), (9, 9), (-13, -5), (-11, -7)]}]","[{'DG': 1, 'NG': 1}]",-2
48,"[1, 0, 2, 0]","[-3, 0, -3, -2, -1]","[1, -2, 3, 3, 3, -6, 9, -12, -19, 20]",6,10,20,"[{'S': 18, 'ψ': [(1, -19), (9, 9), (-12, -6), (20, -2)]}]","[{'DG': 1, 'NG': 1}]",-2


In [86]:
hs3[hs3['n']==11][:2]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
65,"[2, -1, 2, -1]","[-2, -1, -2, 2, 4]","[4, 5, 8, -9, -9, 10, -12, -12, -12, 13, 14]",1,11,14,"[{'S': 18, 'ψ': [(4, 14), (5, 13), (8, 10), (-9, -9)]}]","[{'DG': 1, 'NG': 2}]",-3
66,"[2, 3, 0, -3, 3]","[0, -3, 1, -1, -3]","[1, 1, -2, -3, 6, 6, 6, -7, -10, -10, 12]",15,11,12,"[{'S': 9, 'ψ': [(1, -10), (12, -3), (-7, -2)]}]","[{'DG': 2, 'NG': 0}]",-2


In [87]:
hs3[hs3['n']==12][:5]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
84,"[1, -3, -2, 0, -1]","[-3, -1, -3, -1, -3]","[1, 1, 3, 3, 3, 5, 5, -7, -9, -9, -9, 13]",2,12,13,"[{'S': 6, 'ψ': [(1, 5), (1, -7), (3, 3), (13, -7)]}]","[{'DG': 2, 'NG': 3}]",-5
85,"[1, 0, 2, 0, 2]","[-3, -1, -3, -1, -3]","[7, 9, 9, 9, 11, -13, -13, -27, -27, -27, 31, 31]",2,12,31,"[{'S': 18, 'ψ': [(7, 11), (9, 9), (-13, 31)]}]","[{'DG': 2, 'NG': 3}]",-5
86,"[-2, -3, -2, -3, 2]","[-2, -1, -3, -1, 1]","[1, 1, -2, -2, -2, 6, 6, 7, -8, -8, -9, 10]",4,12,10,"[{'S': 2, 'ψ': [(1, 1), (6, -8), (7, -9), (10, -8)]}]","[{'DG': 2, 'NG': 2}]",-4
87,"[-3, -1, -2, 3, -3]","[-3, -2, -3, -2, 0]","[3, 3, 3, 6, 7, -8, -9, -9, -9, -10, 11, 12]",19,12,12,"[{'S': 18, 'ψ': [(6, 12), (7, 11), (-10, -8), (-9, -9)]}]","[{'DG': 1, 'NG': 3}]",-4
88,"[-3, -1, 2, 0, -3]","[-3, -2, -1, -2, -3]","[3, 4, -8, -9, -9, -9, -10, 14, 15, 15, 15, -21]",12,12,21,"[{'S': 18, 'ψ': [(3, -21), (4, 14), (-10, -8), (-9, -9)]}]","[{'DG': 1, 'NG': 3}]",-4
